In [11]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
import joblib
import keras
import matplotlib.pyplot as plt

In [13]:
TEST_DAYS = 10
PERIOD = '5y'

In [14]:
INDICATOR_DATASET = False

In [15]:
INCLUDE_COMMODITIES = True

In [ ]:
if INDICATOR_DATASET:
    d = joblib.load('nifty_data.pkl')
else:
    d = yf.download(
                tickers="^NSEI",
                period=PERIOD,
                interval='1d',
                progress=False,
                timeout=10
            )
    if INCLUDE_COMMODITIES:
        gold = yf.download(
                    tickers="GC=F",
                    period=PERIOD,
                    interval='1d',
                    progress=False,
                    timeout=10
                ).add_prefix(prefix='gold_')
        crude = yf.download(
                    tickers="CL=F",
                    period=PERIOD,
                    interval='1d',
                    progress=False,
                    timeout=10
                ).add_prefix(prefix='crude_')
        d = pd.concat([d, gold, crude], axis=1)
    
    d['target'] = d.Open/d.Close.shift(-1)
    d.target = d.target.apply(np.floor)

    d['change'] = abs(d['Close'].pct_change(fill_method=None) * 100)

    d['High'] = d['High'].pct_change(fill_method=None) * 100
    d['Low'] = d['Low'].pct_change(fill_method=None) * 100
    d['Open'] = d['Open'].pct_change(fill_method=None) * 100
    d['Close'] = d['Close'].pct_change(fill_method=None) * 100 

    if INCLUDE_COMMODITIES:
        d['gold_High'] = d['gold_High'].pct_change(fill_method=None) * 100
        d['gold_Low'] = d['gold_Low'].pct_change(fill_method=None) * 100
        d['gold_Open'] = d['gold_Open'].pct_change(fill_method=None) * 100
        d['gold_Close'] = d['gold_Close'].pct_change(fill_method=None) * 100

        d['crude_High'] = d['crude_High'].pct_change(fill_method=None) * 100
        d['crude_Low'] = d['crude_Low'].pct_change(fill_method=None) * 100
        d['crude_Open'] = d['crude_Open'].pct_change(fill_method=None) * 100
        d['crude_Close'] = d['crude_Close'].pct_change(fill_method=None) * 100
    # d.rename(columns = {'HighNew':'High','LowNew':'Low','OpenNew':'Open','CloseNew':'Close'}, inplace = True)

    # Remove outliers when Market closes +- 3.5%
    d = d[d['change'] < 3]
    d.dropna(inplace=True)
    d.tail()

In [23]:
def preprocessBeforeScaling(df):
    df['High'] = df['High'].pct_change(fill_method=None) * 100
    df['Low'] = df['Low'].pct_change(fill_method=None) * 100
    df['Open'] = df['Open'].pct_change(fill_method=None) * 100
    df['Close'] = df['Close'].pct_change(fill_method=None) * 100 

    if INCLUDE_COMMODITIES:
        df['gold_High'] = df['gold_High'].pct_change(fill_method=None) * 100
        df['gold_Low'] = df['gold_Low'].pct_change(fill_method=None) * 100
        df['gold_Open'] = df['gold_Open'].pct_change(fill_method=None) * 100
        df['gold_Close'] = df['gold_Close'].pct_change(fill_method=None) * 100

        df['crude_High'] = df['crude_High'].pct_change(fill_method=None) * 100
        df['crude_Low'] = df['crude_Low'].pct_change(fill_method=None) * 100
        df['crude_Open'] = df['crude_Open'].pct_change(fill_method=None) * 100
        df['crude_Close'] = df['crude_Close'].pct_change(fill_method=None) * 100
    
    df = df.ffill().dropna()
    return df

In [ ]:
test_dataset = d.tail(TEST_DAYS)

In [ ]:
d = d[:-(TEST_DAYS+1)]

In [ ]:
if INDICATOR_DATASET:
    x = d.drop(columns=['target'])
    y = d.target
else:
    if INCLUDE_COMMODITIES:
        # x = d.drop(columns=['target', 'Adj Close', 'Volume', 'change', 'gold_Adj Close', 'gold_Volume', 'crude_Adj Close', 'crude_Volume'], errors='ignore')
        x = d.drop(columns=['target', 'Adj Close', 'Volume', 'change', 'gold_Open', 'gold_High', 'gold_Low', 'gold_Adj Close', 'gold_Volume', 'crude_Open', 'crude_High', 'crude_Low', 'crude_Adj Close', 'crude_Volume'], errors='ignore')
    else:
        x = d.drop(columns=['target', 'Adj Close', 'Volume', 'change'], errors='ignore')
    y = d.target

In [ ]:
x

In [ ]:
y

In [ ]:
print('No. of Bullish samples: {}'.format(y[y == 0].size))
print('No. of Bearish samples: {}'.format(y[y == 1].size))

In [ ]:
if not INDICATOR_DATASET:
    print("Using StandardScaler")
    scaler = StandardScaler()
    x = scaler.fit_transform(x.to_numpy())
    x
else:
    print("Using ColumnTransformer")
    col_names = ['Open', 'High', 'Low', 'Close', 'ATR']
    scaler = ColumnTransformer(
        [('StandardScaler', StandardScaler(), col_names)],
        remainder='passthrough'
    )
    x = scaler.fit_transform(x)
x

In [ ]:
visible_devices

In [ ]:
import tensorflow as tf
from keras import Sequential
from keras import Model
from keras.layers import Dense
from keras.optimizers import legacy, SGD
import keras

lr_list = []
def scheduler(epoch, lr):
    if epoch < 2:
        lr = lr
    else:
        lr = lr * tf.math.exp(-0.0025)
    lr_list.append(lr)
    return lr

units = 64 #128 #1024
# sgd = SGD(learning_rate=0.0001, momentum=0.0, nesterov=True)
sgd = legacy.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
kernel_init = 'he_uniform'
activation = 'relu'

callback_mc = keras.callbacks.ModelCheckpoint(
                'best_model.h5',
                verbose=1,
                monitor='val_accuracy',
                save_best_only=True,
                mode='auto'
                )
callback_es = keras.callbacks.EarlyStopping(
                monitor='val_accuracy',
                mode='auto',
                verbose=0,
                patience=100
)
callback_lr = keras.callbacks.LearningRateScheduler(scheduler)

model = Sequential([
    Dense(units, kernel_initializer=kernel_init, activation=activation, input_dim=x.shape[1]),
    # Dense(units, kernel_initializer=kernel_init, activation=activation),
    Dense(units//2, kernel_initializer=kernel_init, activation=activation),
    Dense(units//4, kernel_initializer=kernel_init, activation=activation),
    Dense(units//8, kernel_initializer=kernel_init, activation=activation),
    Dense(units//16, kernel_initializer=kernel_init, activation=activation),
    Dense(units//32, kernel_initializer=kernel_init, activation=activation),
    Dense(1, kernel_initializer=kernel_init, activation='sigmoid'),
])
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
BATCH_SIZE = int(len(y)/6.6125) #128 #24 #4
BATCH_SIZE = 256
print(f'BATCH SIZE = {BATCH_SIZE}')
history = model.fit(x, y, callbacks=[callback_mc, callback_es, callback_lr], batch_size=BATCH_SIZE, epochs=750, validation_split=0.15, verbose=2)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

plt.figure(figsize=(21,6))
plt.rcParams['figure.figsize'] = [8,8]
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'white'

plt.subplot(1, 3, 1)
plt.plot(acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title(f'\nTrain Accuracy: {round(acc[-1],8)}')

plt.subplot(1, 3, 2)
plt.plot(loss, label='Training Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title(f'\nTrain Loss: {round(loss[-1],8)}')
plt.xlabel('epoch')

plt.subplot(1, 3, 3)
plt.plot(lr_list, label='Learning Rate')
plt.legend(loc='upper right')
plt.ylabel('LR')
plt.title(f'\nLearning Rate')
plt.xlabel('epoch')

plt.tight_layout(pad=3.0)
plt.show()

acc = history.history['val_accuracy']
loss = history.history['val_loss']

plt.figure(figsize=(14,6))
plt.rcParams['figure.figsize'] = [8,8]
plt.rcParams['font.size'] = 14
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'white'
plt.subplot(1, 2, 1)
plt.plot(acc, label='Val Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title(f'\nTest Accuracy: {round(acc[-1],8)}')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Val Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title(f'\nTest Loss: {round(loss[-1],8)}')
plt.xlabel('epoch')
plt.tight_layout(pad=3.0)
plt.show()

## Try Realtime Inference

In [26]:
metrics = {
    "TP": 0, "FP": 0, "TN": 0, "FN": 0
}

In [27]:
endpoint = keras.models.load_model('nifty_model_v3.h5')
# endpoint = keras.models.load_model('best_model.h5')
try:
    scaler
except NameError:
    # pkl = joblib.load('nifty_model.pkl')
    pkl = joblib.load('nifty_model_v3.pkl')
    scaler = pkl['scaler']
today = yf.download(
                tickers="^NSEI",
                period=f'{TEST_DAYS}d',
                interval='1d',
                progress=False,
                timeout=10
            )
if INCLUDE_COMMODITIES:
    gold = yf.download(
                    tickers="GC=F",
                    period=f'{TEST_DAYS}d',
                    interval='1d',
                    progress=False,
                    timeout=10
                ).add_prefix(prefix='gold_')
    crude = yf.download(
                tickers="CL=F",
                period=f'{TEST_DAYS}d',
                interval='1d',
                progress=False,
                timeout=10
            ).add_prefix(prefix='crude_')

    today = pd.concat([today, gold, crude], axis=1)
    today = today.drop(columns=['Adj Close', 'Volume', 'gold_Adj Close', 'gold_Volume', 'crude_Adj Close', 'crude_Volume'])
else:
    today = today.drop(columns=['Adj Close', 'Volume'])

###
today = preprocessBeforeScaling(today)
today = today.drop(columns=['gold_Open', 'gold_High', 'gold_Low', 'crude_Open', 'crude_High', 'crude_Low'])
print(today)
###

cnt_correct, cnt_wrong = 0, 0
for i in range(-TEST_DAYS,0):
    try:
        df = today.iloc[i]
        twr = today.iloc[i+1]['Close']
    except IndexError:
        continue
    df = scaler.transform([df])
    pred = endpoint.predict([df], verbose=0)

    if twr > today.iloc[i]['Open']:
        fact = "BULLISH"
    else:
        fact = "BEARISH"

    if pred > 0.5:
        out = "BEARISH"
    else:
        out = "BULLISH"

    if out == fact:
        cnt_correct += 1
        if out == "BULLISH":
            metrics["TP"] += 1
        else:
            metrics["TN"] += 1
    else:
        cnt_wrong += 1
        if out == "BULLISH":
            metrics["FN"] += 1
        else:
            metrics["FP"] += 1

        
    print("{} Nifty Prediction -> Market may Close {} on {}! Actual -> {}, Prediction -> {}, Pred = {}".format(
            today.iloc[i].name.strftime("%d-%m-%Y"),
            out,
            (today.iloc[i].name + pd.Timedelta(days=1)).strftime("%d-%m-%Y"),
            fact,
            "Correct" if fact == out else "Wrong",
            str(np.round(pred[0][0], 2))
            )
        )

print("Correct: {}, Wrong: {}, Accuracy: {}".format(cnt_correct, cnt_wrong, cnt_correct/(cnt_correct+cnt_wrong)))
print(metrics)

                Open      High       Low     Close  gold_Close  crude_Close
Date                                                                       
2023-11-15  0.697093  0.221577  0.441300  0.093698   -0.086659    -2.044465
2023-11-16  0.118561  0.924435  0.241831  0.456152    1.214226    -4.904777
2023-11-17  0.000258 -0.348423  0.206090 -0.168976   -0.115936     4.101506
2023-11-20  0.286664 -0.250181  0.015512 -0.191573   -0.196812     2.253260
2023-11-21  0.201458  0.367730  0.424752  0.453947    1.092183     0.219070
2023-11-22  0.066257 -0.017897 -0.254131  0.143803   -0.395140    -0.861512
2023-11-23  0.224673  0.250180  0.420732 -0.049716   -0.395140    -0.861512
2023-11-24 -0.095063 -0.212833 -0.090467 -0.036869   -0.395140    -0.861512
15-11-2023 Nifty Prediction -> Market may Close BEARISH on 16-11-2023! Actual -> BEARISH, Prediction -> Correct, Pred = 0.59
16-11-2023 Nifty Prediction -> Market may Close BULLISH on 17-11-2023! Actual -> BEARISH, Prediction -> Wrong, Pred

## Save Model for Screeni-py integration

In [ ]:
pkl = {
    #  'model': model,
    'scaler': scaler,
    'columns': ['Open', 'Close', 'High', 'Low', 'gold_Close', 'crude_Close']
}

joblib.dump(pkl, 'nifty_model.pkl')

In [ ]:
pkl = joblib.load('nifty_model_v3.pkl')
z = yf.download(
                tickers="^NSEI",
                period='5d',
                interval='1d',
                progress=False,
                timeout=10
            )
if INCLUDE_COMMODITIES:
    gold = yf.download(
                tickers="GC=F",
                period='5d',
                interval='1d',
                progress=False,
                timeout=10
            ).add_prefix(prefix='gold_')
    crude = yf.download(
                tickers="CL=F",
                period='5d',
                interval='1d',
                progress=False,
                timeout=10
            ).add_prefix(prefix='crude_')
    z = pd.concat([z, gold, crude], axis=1)
z = preprocessBeforeScaling(z)
z = z.iloc[-1]
z = z[pkl['columns']]
print(z)
z = pkl['scaler'].transform([z])
endpoint.predict(z)

In [ ]:
pkl['model'].save('nifty_model.h5')

In [ ]:
pkl

In [ ]:
del pkl['model']

In [ ]:
pkl

In [ ]:
def getSigmoidConfidence(x):
    out_min, out_max = 0, 100
    if x > 0.5:
        in_min = 0.50001
        in_max = 1
    else:
        in_min = 0
        in_max = 0.5
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

map_range(0.9633487, 0.5, 1, 0, 100)